In [12]:
import MeCab
from gensim import models
import urllib.request as req
import os.path
import zipfile

mecab = MeCab.Tagger()
mecab.parse("")
model = models.doc2vec.Doc2Vec.load("aozora.2model")

def read_book(url, zipname):
    if not os.path.exists(zipname):
        req.urlretrieve(url, zipname)
    with zipfile.ZipFile(zipname, 'r') as df:
        for namelist in df.namelist():
            with df.open(namelist, 'r') as nf:
                return nf.read().decode("shift-jis")

            
def split_word(file):
    node = mecab.parseToNode(file)
    document = []
    while node is not None:
        fea = node.feature.split(",")[0]
        if fea in ["名詞"]:
            document.append(node.surface)
        elif fea in ["動詩", "形容詞"]:
            document.append(node.feature.split(",")[6])
        node = node.next
    return document

def similar(name, url):
    zipname = url.split("/")[-1]
    file = read_book(url, zipname)
    word = split_word(file)
    vector = model.infer_vector(word)
    print(name + "との類似度が高い３作品は")
    print(model.docvecs.most_similar([vector], topn=3))

similar("宮沢 賢治:よだかの星",
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介:犬と笛",
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("ポー エドガー・アラン:マリー・ロジェエの怪事件",
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar("紫式部:源氏物語 06 末摘花",
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

宮沢 賢治:よだかの星との類似度が高い３作品は
[('宮澤 賢治:やまなし', 0.9883283376693726), ('宮澤 賢治:どんぐりと山猫', 0.9857877492904663), ('宮澤 賢治:セロ弾きのゴーシュ', 0.9809721112251282)]
芥川 龍之介:犬と笛との類似度が高い３作品は
[('芥川 竜之介:鼻', 0.9763780236244202), ('芥川 竜之介:老年', 0.9695495963096619), ('芥川 竜之介:羅生門', 0.9676886200904846)]
ポー エドガー・アラン:マリー・ロジェエの怪事件との類似度が高い３作品は
[('ポー エドガー・アラン:落穴と振子', 0.9118558168411255), ('ポー エドガー・アラン:ウィリアム・ウィルスン', 0.9000227451324463), ('ポー エドガー・アラン:黒猫', 0.8608208894729614)]
紫式部:源氏物語 06 末摘花との類似度が高い３作品は
[('紫式部:源氏物語 04 夕顔', 0.990317702293396), ('紫式部:源氏物語 02 帚木', 0.9895785450935364), ('紫式部:源氏物語 05 若紫', 0.9884728193283081)]
